In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from albumentations.pytorch import ToTensorV2

import albumentations as A
import monai

from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import time
from collections import defaultdict
import copy
import gc

from utils.dataset import EchoDataset
from utils.train import *
from utils.evaluation import *
from utils.optimizer import *
from utils.utils import *

In [3]:
### Logging part
import logging
from datetime import datetime
from pytz import timezone
import sys

import random

def log(CFG):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    def timetz(*args):
        return datetime.now(tz).timetuple()
    tz = timezone('Asia/Seoul')
    logging.Formatter.converter = timetz

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    # log 출력
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    # log를 파일에 출력
    file_handler = logging.FileHandler(os.path.join('saved_model',CFG['exp_name'], f'init_{opt.type}.log'))
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    logger.info(f"python {' '.join(sys.argv)}")
    logger.info("Timezone: " + str(tz))
    logger.info(f"Training Start")
    return logger


In [4]:
def seed_everything(seed: int=0):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything()

## ⚙️ Configuration

In [5]:
view2cls_dict = {
    'PLAX': 3,
    'PSAX': 0,
    '2CH': 2,
    '4CH': 2,
}
class cfg:
    img_size = 256
    view = 'PLAX'
    lr = 1e-3
    scheduler = 'None'
    reducelr_factor = 0.99
    min_lr = 1e-5
    device = torch.device('mps')
    epochs = 50
    patience = 50
    batch_size = 16
    def __getitem__(self,key):
        return getattr(self, key)
    def __setitem__(self,key,value):
        return setattr(self, key, value)
CFG = cfg()
trace_func = print

In [6]:
dataset_dir = 'output'

In [7]:
def show_lvh(sample):
    image = sample['data'].squeeze()
    plt.figure(figsize=(15,10))
    plt.imshow(image, cmap='gray')
    for coor in sample['label']:
        coor = coor.squeeze()
        plt.plot((coor[0], coor[1]),
                (coor[2], coor[3]), linewidth=3)
        plt.scatter((coor[0], coor[1]), (coor[2], coor[3]), )
    plt.title(f"id: {sample['id']} \
                shape: {sample['data'].shape}")
    plt.axis('off')
    plt.show()

## 📀 Load Data

In [8]:
from albumentations.core.transforms_interface import ImageOnlyTransform

class MinMaxNormalize(ImageOnlyTransform):
    """
    Min-max normalization
    """
    def apply(self, img, **param):
        # minmax normalize
        img = (img - img.min()) / (img.max() - img.min())
        return img


In [9]:
train_ts = A.Compose([
    A.Resize(375, 500),
    MinMaxNormalize(p=1),
    # A.OneOf([
    #     A.MedianBlur(blur_limit=3, p=0.1),
    #     A.MotionBlur(p=0.2),
    # ], p=0.2),
    # A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2,
    #                    rotate_limit=10, p=0.2),
    # A.OneOf([
    #     A.OpticalDistortion(p=0.3),
    # ], p=0.2),
    # A.OneOf([
    #     A.GaussNoise(p=0.2),
    #     A.MultiplicativeNoise(p=0.2),
    # ], p=0.2),
    # A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=0,
    #                      val_shift_limit=0.1, p=0.3),
    # ToTensorV2(),
])
test_ts = A.Compose([
                    A.Resize(375, 500),
                    MinMaxNormalize(p=1),
                    # ToTensorV2(),
                    ])


In [10]:
ds = EchoDataset(root=dataset_dir, split='train', transforms=train_ts)
dl = DataLoader(ds, batch_size=1, shuffle=False, )
s = dl.__iter__().__next__()
s

/Users/kyh/Workspace/2022KOHI/lvh_custom/utils/dataset.py:77: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  'label':torch.tensor(label).reshape(-1).to(torch.float32), # flatten 상태의 포인트를 예측하기 위해 reshape -1 를 수행


{'data': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
 
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]),
 'label': tensor([[309.5703, 153.3203, 321.7773, 139.6484, 250.0000, 278.8086, 258.3008,
          262.6953, 258.7891, 261.2305, 309.5703, 153.3203]]),
 'id': [

In [11]:
# for i, sample in enumerate(dl):
#     show_lvh(sample)
#     if i == 5:
#         break

In [12]:
train_ds = EchoDataset(root=dataset_dir, split='train', transforms=train_ts)
valid_ds = EchoDataset(root=dataset_dir, split='val', transforms=test_ts)

train_dl = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=CFG.batch_size//2, shuffle=False)

## 🧮 Load Model

In [26]:
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self, backbone, in_channels, out_channels, ):
        super().__init__()
        # self.input = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=1)
        for param in backbone.features.parameters():
            param.requires_grad = True
        self.backbone = backbone.features
        # self.backbone = nn.Sequential(
        #                               *(list(backbone.children())[:-1]),
        #                               )    
        self.fc1 = nn.Linear(in_features=1024, out_features=512, bias=True)
        self.fc2 = nn.Linear(in_features=512, out_features=out_channels*2*2, bias=True)
    def forward(self, x):
        # x = self.input(x)
        out = self.backbone(x)
        # out = F.relu(out, inplace=False)
        out = F.adaptive_avg_pool2d(out, output_size=(1,1)).view(x.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        # out = F.relu(out)
        return out
    

In [77]:
from utils.train import train_one_epoch, valid_one_epoch

In [78]:

def run_training(model, train_dl, val_dl, optimizer, scheduler, criterion, CFG):
    """
    모델, 데이터를 받아 학습/검증 수행후 결과에 대해 출력 및 모델 저장
    """
    device = CFG['device']
    num_epochs = CFG['epochs']
    start = time.time()
    early_stopping = EarlyStopping(patience=CFG['patience'], verbose=True, trace_func=trace_func)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_mde = np.inf
    best_epoch = -1
    history = defaultdict(list)
    # 에폭만큼 학습 수행
    for epoch in range(1, num_epochs+1):
        gc.collect()
        # 에폭 단위 학습 수행
        train_loss, train_mde = train_one_epoch(model=model, dataloader=train_dl, optimizer=optimizer, scheduler=scheduler, device=device, criterion=criterion, CFG=CFG)
        # 에폭 단위 검증 수행
        val_loss, val_mde = valid_one_epoch(model=model, dataloader=val_dl, device=device, criterion=criterion, CFG=CFG)
        early_stopping(val_loss, model)

        history['Train Loss'].append(train_loss)
        history['Train MDE'].append(train_mde)
        history['Valid Loss'].append(val_loss)
        history['Valid MDE'].append(val_mde)
        trace_func(f"[Epoch {epoch}/{CFG['epochs']}: Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.5f}, Train MDE: {train_mde:.5f}, Val MDE: {val_mde:.5f}")
        
        if scheduler is not None:
            scheduler.step()
        # validation MDE가 개선이 된 경우 모델을 저장
        if val_mde <= best_mde:
            trace_func(f"Valid Score Improved ({best_mde:.4f} ---> {val_mde:.4f})")
            best_mde = val_mde
            best_epoch = epoch
            best_model_wts = copy.deepcopy(model.state_dict())

            model_name = f"best_epoch-{CFG['view']}.pth"
            torch.save(model.state_dict(), os.path.join(f'saved_model/{1.0}', model_name))

            trace_func(f"Model Saved")
        if early_stopping.early_stop:
            trace_func("Early stopping")
            break

    model_name = f"last_epoch-init_{CFG['view']}.pth"
    torch.save(model.state_dict(), os.path.join(f'saved_model/{1.0}', model_name))
    end = time.time()
    time_elapsed = end-start
    trace_func("Trianing complete in {:.0f}h {:.0f}m {:.0f}s".format(time_elapsed//3600, (time_elapsed%3600)//60, (time_elapsed%3600)%60))
    trace_func("Best Val MDE: {:.4f}".format(best_mde))

    model.load_state_dict(best_model_wts)

    return model, history

In [49]:
model = Model(
                torchvision.models.densenet121(weights=False),
                1,
                3
            )
optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG.lr)
criterion = nn.MSELoss()
model.load_state_dict(torch.load('saved_model/1.0/last_epoch-init_PLAX.pth', map_location=torch.device(CFG['device'])))
model.to(CFG['device'])

/Users/kyh/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model(
  (backbone): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1):

In [80]:

model, hist = run_training(model=model.to(CFG['device']),
             train_dl=train_dl, 
             val_dl=valid_dl, 
             optimizer=optimizer, 
             scheduler=fetch_scheduler(CFG, optimizer), 
             criterion=criterion, 
             CFG=CFG)


Valid : 100%|██████████| 141/141 [00:33<00:00,  4.19it/s, valid_loss=550.2304]


Validation loss decreased (inf --> 550.230385))#.  Saving model ...
[Epoch 1/50: Train Loss: 3104.11711, Val Loss: 550.23039, Train MDE: 0.00000, Val MDE: 0.00000
Valid Score Improved (inf ---> 0.0000)


Train :   0%|          | 0/629 [00:00<?, ?it/s]

Model Saved


Valid : 100%|██████████| 141/141 [00:32<00:00,  4.30it/s, valid_loss=1160.7473]


...

EarlyStopping counter: 27 out of 50
[Epoch 49/50: Train Loss: 43.82196, Val Loss: 83.62499, Train MDE: 0.00000, Val MDE: 0.00000
Valid Score Improved (0.0000 ---> 0.0000)


Train :   0%|          | 0/629 [00:00<?, ?it/s]

Model Saved


Valid : 100%|██████████| 141/141 [00:32<00:00,  4.31it/s, valid_loss=100.9438]


EarlyStopping counter: 28 out of 50
[Epoch 50/50: Train Loss: 50.31287, Val Loss: 100.94384, Train MDE: 0.00000, Val MDE: 0.00000
Valid Score Improved (0.0000 ---> 0.0000)
Model Saved
Trianing complete in 17h 3m 34s
Best Val MDE: 0.0000


In [27]:

restore(sample=s, pred=model(s['data'].to(CFG['device'])).detach().cpu())

NameError: name 'model' is not defined

In [32]:
from utils.utils import restore
restore(sample=s, pred=copy.deepcopy(s['label']), order='xyxy'), s['id']

(tensor([[[634., 314., 659., 286.],
          [512., 571., 529., 538.],
          [530., 535., 634., 314.]]]),
 ['0X66EFA90ADBC0BA4F'])

In [29]:
s['data'].squeeze().shape

torch.Size([3, 375, 500])

In [30]:
from utils.utils import distance_error_per_batch
a,b= distance_error_per_batch(s, model(s['data'].to(CFG['device'])).detach().cpu())

NameError: name 'model' is not defined